# Groupby 1: The `GroupBy` object
By the end of this lecture you will be able to:
- group rows on one or more columns
- iterate over groups
- do aggregations on the groups
- do fast-track grouping on a sorted column


In [ ]:
import polars as pl

In [ ]:
csv_file = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

## The `GroupBy` object
Calling the `group_by` method with a column creates a `GroupBy` object

In [ ]:
(
    df
    .group_by("Pclass")
)

We look at some of the methods we can call on the `GroupBy` object below.

> Why group_by and not groupby? The Polars API aims to generally follow conventions from the world of SQL databases rather than precedents set by Pandas. So we get methods named like group_by rather than groupby

## Iterating over groups
We can access the `DataFrame` for each group by looping over a `GroupBy` object.

When we iterate we get the group key and the sub-`DataFrame` for each group.

In this example we print the mean for each group

In [ ]:
for pclass,group_df in df.group_by("Pclass"):
    print(pclass)
    print(group_df.mean())

If we are grouping by multiple columns then the first element is a `tuple` of the group keys

In [ ]:
for (pclass,survived),group_df in df.group_by("Pclass","Survived"):
    print(f"PClass:{pclass},Survived:{survived}")
    print(group_df.mean())

### Ordering of `group_by`
Due to the parallelisation algorithm the order of the groups in the output changes each time we run `group_by`.

To return the same order use the `maintain_order` argument in `groupby`

In [ ]:
(
    df
    .group_by("Pclass",maintain_order=True)
    .agg(
        pl.col("PassengerId").agg_groups()
    )
)

I explored the reason for `group_by` (and related methods) not preserving order by default in this blog post:https://www.rhosignal.com/posts/polars-ordering/

### Streaming groupby on large datasets
We can run `group_by` on large datasets with the default arguments. However, if we set `maintain_order=True` then `group_by` cannot be run for large datasets in streaming mode.

To see this note how the `AGGREGATE` part of this query plan moves outside of the ` --- STREAMING` if you change `maintain_order` from `False` to `True`

In [ ]:
print(
    pl.scan_csv(csv_file)
    .group_by("Pclass",maintain_order=False)
    .agg(
        pl.col("PassengerId").count()
    )
    .explain(streaming=True)
)

We look at `agg` in more detail in the coming lectures.

## Group values
We use `head` to get the first rows in each group.

In this example we return a `DataFrame` with the first 2 rows from each group

In [ ]:
(
    df
    .group_by("Pclass")
    .head(2)
)

## Simple aggregations
In eager mode we can call aggregations directly on the `GroupBy`

In this example, we count the number of rows per group and we get a single column of counts

In [ ]:
(
    df
    .group_by("Pclass")
    .count()
)

In this example we get the `mean` of all columns

In [ ]:
(
    df
    .group_by("Pclass")
    .mean()
)

We use the `agg` method for more flexibility in the next lecture. 

The methods we can all on `GroupBy` in eager mode are:
 - `first` get the first element of each group
 - `last` get the last element of each group
 - `n_unique` get the number of unique elements in each group
 - `count` get the number of elements in each group
 - `sum` sum the elements in each group
 - `min` get the smallest element in each group
 - `max` get the largest element in each group
 - `mean` get the average of elements in each group
 - `median` get the median in each group
 - `quantile` calculate quantiles in each group
 

## Groupby on multiple columns

We can do `group_by` on multiple columns by passing a list of column names

In [ ]:
(
    df
    .group_by(["Pclass","Survived"])
    .mean()
)

In the following lecture we see how we can also pass expressions to `group_by`

## Groupby on a sorted column
In the lecture "Sorting and Fast-track algorithms" in the Selecting columns and transforming dataframes section we saw how Polars can use fast-track algorithms on sorted columns - if it knows the column is sorted.

A fast-track algorithm can also be used if the groupby column is sorted. See Exercise 3 for an example of this (make sure you have done the Sorting and Fast-track algorithms lecture first).

## Exercises
In the exercises you will develop your understanding of:
- creating a `GroupBy` object
- accessing data from the groups
- aggregating each group
- the effect of the fast-track algorithm on a sorted column

### Exercises 1
Create a `GroupBy` object by grouping on the `Survived` column

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

Create a `DataFrame` showing the row indexes for each group

Create a `DataFrame` with the *last* three rows from each group. Ensure the order of the `DataFrame` is the same each time you run the code.

Get the maximum value for each column in each group

### Exercise 2
We create a `DataFrame` from the Spotify data

In [ ]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(10)
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"
spotify_df = pl.read_csv(spotify_csv,try_parse_dates=True)
spotify_df.head(3)

Format the floating point values so that large floating point numbers are seperated by a comma (or your preferred thousand separator). If you have not encountered this before try tab-completing the following cell to find an appropriate method

In [ ]:
pl.Config.set

Group by the `artist` and `title` columns and get the maximum of the other columns. Sort the output with the largest values of streams first

In [ ]:
(
    spotify_df
    <blank>
    .head()
)

It's easy to forget that the max values are not set by the `stream` columns but all come from different rows. For example the max streams value for each of these entries would have been 1 but instead we see the lowest rank for the track in this output

We want to inspect some data for the top-streaming artists by printing it out:
- filter `spotify_df` to include only rows that had more than 10 million streams
- `group_by` the `artist` column
- ensure the order of the output is the same each time
- print the `artist` key
- print the sub-`DataFrame`

Repeat this exercise but in this case grouping by the `artist` and `title` column and printing the artist and title for each group

### Exercise 3
We look at the effect of sorting and the fast-track algorithm on a `group_by` operation.

We create a `DataFrame` with an `id` column of integers and a `values` column

- The `N` variable sets the number of rows in the `DataFrame`
- The `cardinality` sets the number of distinct group keys in the `id` column

We begin with a low cardinality and see the effect of increasing the cardinality later in the exercise.

We pre-sort the `id`s before creating the `DataFrame`

In [ ]:
pl.Config.set_tbl_rows(4)
import numpy as np
np.random.seed(0)
N = 10_000_000
cardinality = 10
# Create a sorted array of id integers
sorted_array = np.sort(np.random.randint(0,cardinality,N))
df = (
    pl.DataFrame(
        {
            "id":[i for i in sorted_array],
            "values":np.random.standard_normal(N)
        }
    )
)
df.head(3)

Time how long it takes to groupby the `id` column and take the mean of the `values` column without any fast-track algorithm

In [ ]:
%%timeit -n1 -r3
(
    df
    <blank>
)

Create a new `DataFrame` called `df_sorted` where we tell Polars the `id` column is sorted

In [ ]:
df_sorted = (
    df
    <blank>
)
df_sorted["id"].flags

Time how long it takes to groupby the `id` column and take the mean of the `values` column **with** a fast-track algorithm

In [ ]:
%%timeit -n1 -r3
(
    df_sorted
    <blank>
)

Compare the difference between the sorted and non-sorted algorithms when the cardinality of `id` is higher. Try:
- `cardinality = 1_000` and 
- `cardinality = 1_000_000`


## Solutions

### Solutions to Exercise 1
Create a `GroupBy` object by grouping on the `Survived` column

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Survived")
)

Create a `DataFrame` showing the row indexes for each group

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Survived")
    .agg(
        pl.col("PassengerId").agg_groups()
    )
)

Create a `DataFrame` with the *last* three rows from each group. Ensure the order of the `DataFrame` is the same each time you run the code.

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Survived",maintain_order=True)
    .tail(3)
)

Get the maximum value for each column in each group

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Survived",maintain_order=True)
    .max()
)

### Solutions to Exercise 2
We create a `DataFrame` from the Spotify data

In [ ]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(10)
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"
spotify_df = pl.read_csv(spotify_csv,try_parse_dates=True)
spotify_df.head(3)

Format the floating point values so that large floating point numbers are seperated by a comma (or your preferred thousand separator). If you have not encountered this before try tab-completing the following cell to find an appropriate method

In [ ]:
pl.Config.set_thousands_separator(",")

Group by the `artist` and `title` columns and get the maximum of the other columns. Sort the output with the largest values of streams first

In [ ]:
(
    spotify_df
    .group_by("artist","title")
    .max()
    .sort("streams",descending=True)
    .head()
)

It's easy to forget that the max values are not set by the `stream` columns but all come from different rows. For example the max streams value for each of these entries would have been 1 but instead we see the lowest rank for the track in this output

We want to inspect some data for the top-streaming artists by printing it out:
- filter `spotify_df` to include only rows that had more than 10 million streams
- `group_by` the `artist` column
- ensure the order of the output is the same each time
- print the `artist` key
- print the sub-`DataFrame`

In [ ]:
for artist,artist_df in (
    spotify_df
    .filter(
        pl.col("streams") > 10_000_000,
    )
    .group_by("artist",maintain_order=True)
):
    print(artist)
    print(artist_df)


Repeat this exercise but in this case grouping by the `artist` and `title` column and printing the artist and title for each group

In [ ]:
for (artist,title),artist_df in (
    spotify_df
    .filter(
        pl.col("streams") > 10_000_000,
    )
    .group_by("artist","title",maintain_order=True)
):
    print(artist,title)
    print(artist_df)


### Solution to exercise 3
We look at the effect of sorting on a `groupby` operation.

We create a `DataFrame` with an `id` column of integers and a `values` column

- The `N` variable sets the number of rows in the `DataFrame`
- The `cardinality` sets the number of distinct `id`s

We pre-sort the `id`s before creating the `DataFrame`

In [ ]:
pl.Config.set_tbl_rows(4)
import numpy as np
np.random.seed(0)
N = 10_000_000
cardinality = 1_000_000
# Create a sorted array of id integers
sorted_array = np.sort(np.random.randint(0,cardinality,N))
df = (
    pl.DataFrame(
        {
            "id":[i for i in sorted_array],
            "values":np.random.standard_normal(N)
        }
    )
)
df.head(3)

Time how long it takes to groupby the `id` column and take the mean of the `values` column without any fast-track algorithm

In [ ]:
%%timeit -n1 -r3
(
    df
    .group_by("id")
    .agg(
        pl.col("values").mean()
    )
)

Create a new `DataFrame` called `df_sorted` where we tell Polars the `id` column is sorted

In [ ]:
df_sorted = (
    df
    .with_columns(
        pl.col("id").set_sorted()
    )
)
df_sorted["id"].flags

Time how long it takes to groupby the `id` column and take the mean of the `values` column **with** a fast-track algorithm

In [ ]:
%%timeit -n1 -r3
(
    df_sorted
    .group_by("id")
    .agg(
        pl.col("values").mean()
    )
)

Compare the difference in timings between the standard and fast-track algorithm when the cardinality of `id` is higher


The difference is much smaller (and possibly negative) when the cardinality of `id` is high